# Tutorial

In this tutorial we fairly compare a number of ensemble methods using EI's built in nested cross-validation implementation, and show how predictions can be made with the selected final model. We then show how we can intepret the model by calculating feature rankings.

### Performance analysis and selection of ensemble methods

First of all let's import some `sklearn` models, `EnsembleIntegration` and some additional ensemble methods:

In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.ei import EnsembleIntegration
from eipy.additional_ensembles import MeanAggregation, CES

Next make some dummy "multi-modal" data from the breast cancer dataset:

In [49]:
import numpy as np
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
feature_names = data.feature_names
X = data.data
y = np.abs(data.target - 1)  # make "malignancy" the positive class rather than "benign"

X_1 = X[:, 0:10]
X_2 = X[:, 10:]

Create a dictionary containing data modalities:

In [51]:
data = {
                "Modality_1": X_1,
                "Modality_2": X_2
                }

Define base predictors:

In [52]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(), 
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True),
}

Initialise Ensemble Integration:

In [53]:
EI = EnsembleIntegration(base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy="undersampling",
                        sampling_aggregation="mean",
                        n_jobs=-1,
                        random_state=38,
                        project_name="breast_cancer",
                        model_building=True,
                        )

Train base predictors on each modality. Remember to include the unique modality name.

In [54]:
for name, modality in data.items():
    EI.train_base(modality, y, modality=name)

Training base predictors on Modality_1...

... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_2...

... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


We can check the cross validated performance of each base predictor on each modality with `base_summary`:

In [56]:
EI.base_summary['metrics']

modality        Modality_1                                                    \
base predictor        ADAB        DT        GB       KNN        LR       MLP   
fmax (minority)   0.920188  0.868009  0.929577  0.836879  0.893617  0.841346   
f (majority)      0.952247  0.914616  0.957865  0.903497  0.937063  0.908587   
AUC               0.984515  0.900124  0.983048  0.925209  0.973125  0.936658   
max MCC           0.864439  0.785814  0.887469  0.753639  0.826775  0.754265   

modality                                                Modality_2            \
base predictor         NB        RF       SVM       XGB       ADAB        DT   
fmax (minority)  0.907449  0.928910  0.841558  0.933025   0.947619  0.910755   
f (majority)     0.941007  0.958101  0.918991  0.958865   0.969359  0.944365   
AUC              0.975781  0.984851  0.937173  0.983550   0.985955  0.932925   
max MCC          0.850729  0.887975  0.771895  0.888915   0.909783  0.856198   

modality                                                                     \
base predictor         GB       KNN        LR       MLP        NB        RF   
fmax (minority)  0.964706  0.882206  0.941725  0.908189  0.934272  0.957746   
f (majority)     0.978962  0.936401  0.964739  0.949660  0.960674  0.974719   
AUC              0.990341  0.961253  0.989839  0.968884  0.990381  0.991022   
max MCC          0.943675  0.822812  0.906350  0.856887  0.894973  0.932493   

modality                             
base predictor        SVM       XGB  
fmax (minority)  0.887218  0.963680  
f (majority)     0.939107  0.979310  
AUC              0.969254  0.994305  
max MCC          0.830873  0.943838

Now let's define some meta models for stacked generalization. We add an "S." prefix to the keys of stacking algorithms.

In [57]:
meta_predictors = {     
                    'Mean' : MeanAggregation(),
                    'CES' : CES(),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(), 
                    'S.GB': GradientBoostingClassifier(),
                    'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}

Train meta models:

In [58]:
EI.train_meta(meta_predictors=meta_predictors)

Analyzing ensembles: |██████████|100%
Training final meta models: |██████████|100%


Check the meta summary with `meta_summary`:

In [59]:
EI.meta_summary['metrics']

,Mean,CES,S.ADAB,S.XGB,S.DT,S.RF,S.GB,S.KNN,S.LR,S.NB,S.MLP,S.SVM
fmax (minority),0.959233,0.959036,0.957547,0.961722,0.947867,0.966346,0.964029,0.956311,0.966507,0.964200,0.964029,0.968974
f (majority),0.976422,0.976487,0.974790,0.977778,0.969274,0.980609,0.979196,0.975207,0.980556,0.979138,0.979196,0.981919
AUC,0.991081,0.991425,0.985155,0.989879,0.957693,0.990725,0.988393,0.985664,0.991729,0.984964,0.990553,0.986629
max MCC,0.935993,0.932354,0.928653,0.935993,0.917168,0.947402,0.943565,0.932522,0.947402,0.922254,0.943674,0.951067


The SVM stacking algorithm has the best $\text{F}_\text{max}$ performance so let's select it as our final model. Since we ran EI with `model_building=True`, we can now predict. Let's just predict the training data, and apply the $\text{F}_\text{max}$ training threshold:

In [60]:
y_pred = EI.predict(X_dict=data, meta_model_key='S.LR')

threshold = EI.meta_summary['thresholds']['S.LR']['fmax (minority)']

y_pred[y_pred>=threshold] = 1
y_pred[y_pred<threshold] = 0

print(y_pred)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1.
 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1.
 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1.


### Interpreting the final model

We now use `PermutationInterpreter` to interpret the final SVM stacked generalization model. Let's first import the class and our chosen metric, and initialise the interpreter:

In [85]:
from eipy.interpretation import PermutationInterpreter
from eipy.utils import f_minority_score

interpreter = PermutationInterpreter(EI=EI,
                                     metric=f_minority_score,
                                     meta_predictor_keys=['S.SVM'])

Calculate feature importance scores:

In [86]:
interpreter.rank_product_score(X_dict=data, y=y)

Interpreting ensembles...



Calculating local feature ranks: |██████████|100%


complete!


Calculating local model ranks: |██████████|100%

complete!
Calculating combined rank product score...
... complete!


We can now inspect the most important features for model prediction:

In [95]:
ranking_dataframe = interpreter.ensemble_feature_ranking['S.SVM']
reordered_feature_names = feature_names[ranking_dataframe.index]
ranking_dataframe['feature'] = reordered_feature_names

In [96]:
ranking_dataframe

,modality,feature,RPS,feature rank,ensemble method
3,Modality_1,mean area,0.086000,1.0,S.SVM
13,Modality_2,area error,0.103750,2.0,S.SVM
22,Modality_2,worst perimeter,0.110250,3.0,S.SVM
23,Modality_2,worst area,0.120750,4.0,S.SVM
21,Modality_2,worst texture,0.174500,5.0,S.SVM
27,Modality_2,worst concave points,0.177125,6.0,S.SVM
1,Modality_1,mean texture,0.202000,7.0,S.SVM
7,Modality_1,mean concave points,0.202250,8.0,S.SVM
20,Modality_2,worst radius,0.209250,9.0,S.SVM
26,Modality_2,worst concavity,0.219375,10.0,S.SVM
